In [1]:
import warnings
warnings.filterwarnings('ignore')

import joblib
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
df = pd.read_csv('data_clean.csv')

In [3]:
import torch
from sklearn.preprocessing import StandardScaler
from model_definition import RegressionNN # Make sure the class is imported

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the checkpoint using the "easy fix"
checkpoint = torch.load(
    'model_checkpoint.pth',
    map_location=device,
    weights_only=False # Disables the security check
)

# The rest of your code remains the same
model = RegressionNN(input_dim=checkpoint['input_dim'], output_dim=checkpoint['output_dim'])
model.load_state_dict(checkpoint['model_state_dict'])
X_scaler = checkpoint['x_scaler']
y_scaler = checkpoint['y_scaler2']
model.to(device)
model.eval()

RegressionNN(
  (net): Sequential(
    (0): Linear(in_features=17, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [4]:
# Prepare features and targets
features = ['pred_x_km', 'pred_y_km', 'pred_z_km', 'pred_vx_km_s', 'pred_vy_km_s', 'pred_vz_km_s',
            'KP_SUM_mean', 'AP_AVG_mean', 'F10.7_ADJ_mean', 'F10.7_ADJ_LAST81_mean', 'F10.7_OBS_mean',
            'ballistic_coefficient', 'grav_potential', 'grav_x', 'grav_y', 'grav_z', 'grav_magnitude']
targets = ['real_x_km', 'real_y_km', 'real_z_km', 'real_vx_km_s', 'real_vy_km_s', 'real_vz_km_s']

X = X_scaler.transform(df[features].values)
y = y_scaler.transform(df[targets].values)

# Predict
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()
with torch.no_grad():
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    preds = model(X_tensor).cpu().numpy()

In [ ]:
preds

array([[-1.3222188 , -0.29914677,  1.2096474 ,  0.3641316 , -1.5319557 ,
        -0.58289117],
       [-1.3096524 , -0.32264176,  1.2269927 ,  0.38155177, -1.522002  ,
        -0.58560085],
       [ 0.53021806, -1.2482924 , -0.48438838,  1.4623244 ,  0.61905384,
        -0.6003745 ],
       ...,
       [ 0.711335  ,  1.1502031 , -0.77921325, -1.3242419 ,  0.8325634 ,
        -0.5567477 ],
       [ 0.8479332 ,  1.0770451 , -0.9090788 , -1.2428906 ,  0.99305665,
        -0.5596014 ],
       [ 0.90756893,  1.0030826 , -0.9535518 , -1.1570039 ,  1.0626267 ,
        -0.56089854]], shape=(10000, 6), dtype=float32)

In [6]:
pd.DataFrame(preds, columns=['real_x_km', 'real_y_km', 'real_z_km',
            'real_vx_km_s', 'real_vy_km_s', 'real_vz_km_s'])

,real_x_km,real_y_km,real_z_km,real_vx_km_s,real_vy_km_s,real_vz_km_s
0,-1.322219,-0.299147,1.209647,0.364132,-1.531956,-0.582891
1,-1.309652,-0.322642,1.226993,0.381552,-1.522002,-0.585601
2,0.530218,-1.248292,-0.484388,1.462324,0.619054,-0.600375
3,0.394795,-1.317330,-0.367639,1.535507,0.463790,-0.602557
4,0.420083,-1.308774,-0.378659,1.522825,0.493349,-0.604267
...,...,...,...,...,...,...
9995,0.481800,1.269152,-0.549065,-1.465602,0.563611,-0.554859
9996,0.588118,1.226427,-0.649040,-1.412615,0.686945,-0.557056
9997,0.711335,1.150203,-0.779213,-1.324242,0.832563,-0.556748
9998,0.847933,1.077045,-0.909079,-1.242891,0.993057,-0.559601
